In [1]:
using TensorFlow
using CSV: read
using Distributions: Normal
using ProgressMeter: @showprogress

In [2]:
# Preprocessing the data
f = CSV.read("./gyroscope.csv")

T = length(f[:,11])
Nw = 3                          # Window size
Nc = 3                          # number of inputs
Ncw = Nc * Nw                   # input size
TNw = Int64(floor(T/Nw))        # number of time stamps

dim1 =  Int64(floor(Ncw / 2))
dim2 =  Int64(floor(dim1 / 2))

raw_train = Array{Float32, 2}(Nc,T)
raw_train = f[:,11:13]

s = Array{Float32, 2}(Ncw,TNw)
for i=1:TNw
    s[:,i] .= vec(convert(Array,f[Nw*(i-1)+1:Nw*i,11:13]))
end
s

9×8127 Array{Float32,2}:
 -0.0082558   -0.0066579   -0.00292948  …  -0.00798948  -0.00878843
 -0.00532632  -0.00585895  -0.00292948     -0.00852212  -0.0111853 
 -0.00346211  -0.00452737  -0.00692422     -0.00878843  -0.0111853 
  0.0215716    0.0149137    0.0218379       0.0218379    0.0218379 
  0.0205063    0.0143811    0.0186421       0.021039     0.0221042 
  0.0191748    0.0191748    0.0125169   …   0.0215716    0.0229032 
 -0.00186421  -0.00239685  -0.00692422     -0.00452737  -0.00479369
 -0.0015979   -0.0015979   -0.00905475     -0.00372843  -0.00292948
 -0.00292948  -0.00532632  -0.00878843     -0.00559264  -0.00213053

In [3]:
function weight_variable(shape, λ=0.1, reg=nn.l2_loss)
    W = Variable(map(Float64, rand(Normal(0, .01), shape...)))
    # regularization term
    wd = λ * reg(W)
    TensorFlow.add_to_collection(:regularizers, wd)
    return W
end

function bias_variable(shape)
    b = Variable(fill(Float64(.1), shape...))
    return b
end

# returns a tensor that computes the autoencoder output for a timestamp t of the input array s
function autoencoder(input)
    enc = nn.sigmoid(W2 * nn.sigmoid(W1 * input + b1) + b2)
    TensorFlow.add_to_collection(:encoders, enc)
    return nn.sigmoid(W1t * nn.sigmoid(W2t * enc + b3) + b4)
end

autoencoder (generic function with 1 method)

In [4]:
g = Graph()
sess = Session(g)

@tf begin
    
    x = placeholder(Float64, shape=[Ncw, 1])
        
    W1 = weight_variable([dim1, Ncw])
    b1 = bias_variable([dim1, 1])

    W2 = weight_variable([dim2, dim1])
    b2 = bias_variable([dim2, 1])

    # this is a tied weight
    W2t = transpose(W2)
    b3 = bias_variable([dim1, 1])

    # this is a tied weight too
    W1t = transpose(W1)
    b4 = bias_variable([Ncw, 1])
    
end

2018-07-10 17:12:30.510308: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.2 AVX AVX2 FMA


TensorFlow.Variables.Variable{Float64}(<Tensor node_6:1 shape=(9, 1) dtype=Float64>, <Tensor node_6/Assign:1 shape=unknown dtype=Float64>)

In [5]:
# initialize input as a matrix; passing vectors does not work
input = Array{Any, 2}(Ncw,1)

@tf begin
    data = constant(s)

    # iterate over all time windows
    @showprogress for i=1:5 #TNw
        # input is the i-th column of the data
        input = slice(data,[1,i],[Ncw,1])
        TensorFlow.add_to_collection(:losses, reduce_mean((autoencoder(input) - input)^2 ))
    end
    # add MSE and regularization loss for total loss
    loss = reduce_sum(stack(get_collection(:losses))) + reduce_sum(stack(get_collection(:regularizers)))  
    
    
    # iterate over all time windows again; calling get_collection before a collection has been
    # populated does not give a reference to the Array that will be populated by add_to_collection
    code = get_collection(:encoders)
    @showprogress for i=2:5 #TNw
        TensorFlow.add_to_collection(:dists, nn.l2_loss(code[i]-code[i-1])/sqrt(multiply(nn.l2_loss(code[i]),nn.l2_loss(code[i-1]))))
    end  
    # concatenate encoder outputs to build the code
    dist = stack(get_collection(:dists))'
    
    opt = train.AdamOptimizer(1e-4)
    optimizer = train.minimize(opt, loss)
    
    saver = train.Saver()
    save_path = "./checkpoints/tensorflow"
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


"./checkpoints/tensorflow"

In [ ]:
# train new network
run(sess, global_variables_initializer())

emax = 1e4
@showprogress for epoch=1:emax
    cur_loss, _ = run(sess, (loss, optimizer))
    if epoch%(emax/10) == 0
        info("\nepoch: $epoch | current loss: $cur_loss\n")
    end
end

train.save(saver, sess, save_path)

In [38]:
# reuse weights from a previously stored network
run(sess, global_variables_initializer())
train.restore(saver, sess, save_path)

emax = Int32(1e4)
elapsed = 0; tic()
for epoch=1:emax
    cur_loss, _ = run(sess, (loss, optimizer))
    cur_loss = Float32(cur_loss)
    if epoch%(emax/10) == 0
        elapsed += toq(); tic()
        info("epoch: $(@sprintf("%5i", epoch)) | elapsed time: $(@sprintf("%6.1f", elapsed)) seconds | loss: $(@sprintf("%.6f", cur_loss)) | accuracy: not implemented")
    end
end

INFO: epoch:  1000 | elapsed time:    1.18 seconds | loss: 0.243071 | accuracy: not implemented
INFO: epoch:  2000 | elapsed time:    2.27 seconds | loss: 0.216937 | accuracy: not implemented
INFO: epoch:  3000 | elapsed time:    3.33 seconds | loss: 0.193763 | accuracy: not implemented
INFO: epoch:  4000 | elapsed time:    4.40 seconds | loss: 0.173015 | accuracy: not implemented
INFO: epoch:  5000 | elapsed time:    5.44 seconds | loss: 0.154339 | accuracy: not implemented
INFO: epoch:  6000 | elapsed time:    6.48 seconds | loss: 0.137487 | accuracy: not implemented
INFO: epoch:  7000 | elapsed time:    7.54 seconds | loss: 0.122275 | accuracy: not implemented
INFO: epoch:  8000 | elapsed time:    8.60 seconds | loss: 0.108548 | accuracy: not implemented
INFO: epoch:  9000 | elapsed time:    9.63 seconds | loss: 0.096177 | accuracy: not implemented
INFO: epoch: 10000 | elapsed time:   10.69 seconds | loss: 0.085044 | accuracy: not implemented


In [12]:
run(sess,dist)

4-element Array{Float64,1}:
 3.5845e-12 
 8.58657e-13
 3.47217e-11
 2.12193e-11